---
# Section 1.3: Triangular Systems
---

Let $L \in \mathbb{R}^{n \times n}$ be a **lower-triangular matrix** and $U \in \mathbb{R}^{n \times n}$ be an **upper-triangular matrix**. Then,

$$
L = 
\begin{bmatrix}
l_{11} \\
\vdots & \ddots \\
l_{n1} & \cdots & l_{nn}
\end{bmatrix}
\qquad \text{and} \qquad
U = 
\begin{bmatrix}
u_{11} & \cdots & u_{1n} \\
& \ddots & \vdots \\
& & u_{nn}
\end{bmatrix}.
$$

> ### Theorem:
> Let $G$ be an $n \times n$ triangular matrix. Then
> 
> $$ 
\text{$G$ is nonsingular} \quad \Longleftrightarrow \quad g_{ii} \neq 0, \quad i = 1,\ldots,n.
$$

---
### Exercise: 

Prove the above theorem.

### Proof:

Since $G$ is triangular, $\det(G) = g_{11} g_{22} \cdots g_{nn}$. Therefore, $\det(G) \ne 0$ if and only if $g_{ii} \ne 0$ for $i = 1,\ldots,n$. $\blacksquare$

---

### Exercise:
Solve the following lower-triangular system $Lx = b$ and use `Julia` to check your answer.

$$
\begin{bmatrix}
2 & 0 & 0 & 0 \\
-1 & 2 & 0 & 0 \\
3 & 1 & -1 & 0 \\
4 & 1 & -3 & 3
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4
\end{bmatrix} =
\begin{bmatrix}
2 \\ 3 \\ 2 \\ 9
\end{bmatrix}.
$$

$2 x_1 = 2 \implies x_1 = 1$

$-x_1 + 2x_2 = 3 \implies -1 + 2x_2 = 3 \implies 2x_2 = 4 \implies x_2 = 2$

$3x_1 + x_2 - x_3 = 2 \implies 3 + 2 - x_3 = 2 \implies -x_3 = -3 \implies x_3 = 3$

$4x_1 + x_2 - 3x_3 + 3x_4 = 9 \implies 4 + 2 - 9 + 3x_4 = 9 \implies 3x_4 = 12 \implies x_4 = 4$

Therefore,

$$
x =
\begin{bmatrix}
1 \\ 2 \\ 3 \\ 4
\end{bmatrix}.
$$

In [ ]:
L = [
    2 0 0 0
    -1 2 0 0
    3 1 -1 0
    4 1 -3 3.0
]
b = [2, 3, 2, 9.0]
x = L\b

In [ ]:
using LinearAlgebra

In [ ]:
istril(L)

In [ ]:
M = LowerTriangular(L)

In [ ]:
istril(M)

---

In general, the solution of a lower-triangular system $Lx = b$ is given by

$$
x_i = \left( b_i - \sum_{j=1}^{i-1} l_{ij}x_j \right) \bigg/ l_{ii}, \quad i = 1, \ldots, n.
$$

---

### Exercise:

Write a `Julia` function, `x = row_fs(L, b)`, that returns the solution $x$ of a lower-triangular system $Lx = b$ using _forward substitution_. Make sure to include a check that the matrix $L$ is nonsingular. Test your code for accuracy and efficiency.

---

## Column-oriented forward substitution

Notice that the algorithm above is **row-oriented**, which is good for computer languages that store matrices in row-major order (like C/C++ or Python), but is bad for column-major order languages (like Fortran, MATLAB, or Julia).

We can produce a forward-substitution algorithm that is **column-oriented** by using the concept of **block-matrix multiplication**.

---

### Exercise:

Let the lower-triangular linear system $Lx = b$ be partitioned as

$$
\begin{bmatrix}
l_{11} & 0 \\
\hat{l} & \hat{L} \\
\end{bmatrix}
\begin{bmatrix}
x_1 \\ \hat{x}
\end{bmatrix} = 
\begin{bmatrix}
b_1 \\ \hat{b}
\end{bmatrix}
$$

where $\hat{L}$ is an $(n-1) \times (n-1)$ *lower-triangular* matrix and $\hat{x}$ and $\hat{b}$ are vectors of length $n - 1$.

1. Perform block-matrix multiplication.

2. Determine a _recursive algorithm_ for solving the linear system $Lx = b$.

3. Implement your recursive algorithm in `Julia` (name your function `col_rfs`).

4. Test your code for accuracy and efficiency.

---

### Exercise:

Use column-oriented forward substitution to solve the lower-triangular system.

$$
\begin{bmatrix}
2 & 0 & 0 & 0 \\
-1 & 2 & 0 & 0 \\
3 & 1 & -1 & 0 \\
4 & 1 & -3 & 3
\end{bmatrix}
\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4
\end{bmatrix} =
\begin{bmatrix}
2 \\ 3 \\ 2 \\ 9
\end{bmatrix}
$$


---

### Exercise:

Inspired by your calculations in the previous exercise, write a non-recursive column-oriented forward-substitution code in `Julia` (name your function `col_fs`). Test your function for accuracy and efficiency.

---

### Exercise:

How do your `row_fs` and `col_fs` functions compare to `Julia`'s built-in `x = L\b`?

---

### Exercise:

Determine the total number of flops to solve an $n \times n$ triangular linear system using two different methods:

1. Directly count the flops in your nonrecursive code.
2. Count the flops using the recursive version of your code.

---

### Exercise:

Write a `Julia` function `col_fs!(b, L)` that **overwrites** the array `b` with the solution of the linear sytem $Lx = b$. Test the efficiency of your function.

---